In [ ]:
import re
import nltk
import numpy as np
from keras.preprocessing.text import one_hot
from sklearn import svm
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelBinarizer

In [ ]:
#Function to extract adjectives from the contents. Return a list of strings
#Need nltk library: import nltk
def extract_adjective(sentences):
    adj_sentences = list()
    for sentence in sentences:
        words = nltk.word_tokenize(sentence)
        adj_tags = nltk.pos_tag(words)
        one_adj_sentence = ""
        for index, tag in enumerate(adj_tags, start = 0):
            one_tag = tag[1]
            if one_tag in ['JJ', 'JJR', 'JJS']:
                one_adj_sentence += words[index]
                one_adj_sentence += " "
        adj_sentences.append(one_adj_sentence)
        print(one_adj_sentence)
    return adj_sentences


In [ ]:
def clean_sentence(s):
    c = s.lower().strip()
    return re.sub('[^a-z ]', '', c)

In [ ]:
def encode_words(sentences):
    encoded = list()
    for sentence in sentences:
        words = list()
        mapping = list()
        words = nltk.word_tokenize(sentence)
        for word in words:
            mapping.append(one_hot(word,30000)[0])
        encoded.append(mapping);
    return encoded

In [ ]:
def append_with_max_len(encoded,length):
    for e in encoded:
        for count in range(len(e), length):
            e.append(0)
    return encoded; 
    

In [ ]:
def set_label(fake_size, real_size):
    label = list()
    for counter in range(0,fake_size):
        label.append(0)
    for counter in range(0,real_size):
        label.append(1)
    return label

In [ ]:
train_news = list()
fake_size = 0
real_size = 0
with open('./data/fake_news_training.txt') as train1:
    with open('./data/real_news_training.txt') as train2:
        for line in train1:
            train_news.append(clean_sentence(line))
            fake_size = fake_size+1
        for line in train2:
            train_news.append(clean_sentence(line))
            real_size = real_size+1
predict_news = list()
with open('./data/testing_real.txt') as predict1:
    with open('./data/testing_fake.txt') as predict2:
        for line in predict1:
            predict_news.append(clean_sentence(line))
        for line in predict2:
            predict_news.append(clean_sentence(line))
words = encode_words(extract_adjective(train_news))    
print(words)
labels = set_label(fake_size, real_size)
print(len(words))
print(len(labels))
appended = append_with_max_len(words,len(max(words, key=len)))
print(appended)
predict_words = encode_words(predict_news)
classif = OneVsRestClassifier(estimator=SVC(random_state=0))
predict_words = encode_words(extract_adjective(predict_news))
classif.fit(appended, labels).predict(append_with_max_len(predict_words,len(max(words, key=len))))
